# INSTRUCTIONS FOR FORMATTING THE EXPERIMENTAL DATA

To use our codes with the exoerimental data [1,2] you should first convert the "Exp_GunData4ImagingFD.mat" file to torch tensors and reshape them. 

[1] J. Wang, P. Aubry and A. Yarovoy, "3-D Short-Range Imaging With Irregular MIMO Arrays Using NUFFT-Based Range Migration Algorithm," in IEEE Transactions on Geoscience and Remote Sensing, vol. 58, no. 7, pp. 4730-4742, July 2020, doi: 10.1109/TGRS.2020.2966368.

[2] Jianping Wang, January 10, 2020, "EM data acquired with irregular planar MIMO arrays", IEEE Dataport, doi: https://dx.doi.org/10.21227/src2-0y50.


### Step 1: Download the data from [2] (https://dx.doi.org/10.21227/src2-0y50)
### Step 2: Run the following MATLAB code:


    path_to_experimental_data="..\data\datasets\experimental";
    load(path_to_experimental_data+"Exp_GunData4ImagingFD.mat")
    
    measurements_RTF=EMData.Values;
    T=[MIMOArray.Transmitter.X;MIMOArray.Transmitter.Y;MIMOArray.Transmitter.Z];
    R=[MIMOArray.Receiver.X;MIMOArray.Receiver.Y;MIMOArray.Receiver.Z];
    F=OperateFreq;
    
    save(path_to_experimental_data+"measurements_RTF.mat",'measurements_RTF')
    save(path_to_experimental_data+"T.mat",'T')
    save(path_to_experimental_data+"R.mat",'R')
    save(path_to_experimental_data+"F.mat",'F')

### Step 3: Run the following Python code:

In [26]:
from scipy import io
import torch

Measurements = io.loadmat('data/datasets/experimental/measurements_RTF.mat')
Transmitters = io.loadmat('data/datasets/experimental/T.mat')
Receivers = io.loadmat('data/datasets/experimental/R.mat')
freqs = io.loadmat('data/datasets/experimental/F.mat')

print(type(Measurements['measurements_RTF']))
print(type(Transmitters['T']))
print(type(Receivers['R']))
print(type(freqs['F']))

R = torch.tensor(Receivers['R'])
T = torch.tensor(Transmitters['T'])
F = torch.tensor(freqs['F']).squeeze()
MEAS_NCKTR = torch.transpose(torch.tensor(Measurements['measurements_RTF']),dim0=0,dim1=2)[None,None,...]

print('{:30} {:3}'.format('Number of Transmitters:',T.shape[1]))
print('{:30} {:3}'.format('Number of Receivers:', R.shape[1]))
print('{:30} {:3}'.format('Number of Freqs.:',F.shape[0]))
print('{:30} {}'.format('Measurement Shape (NCktr):',tuple(MEAS_NCKTR.shape)))

tx_positions = torch.zeros_like(T.T)
tx_positions[:,0] = T[0,:]
tx_positions[:,1] = T[2,:]
tx_positions[:,2] = T[1,:]

rx_positions = torch.zeros_like(R.T)
rx_positions[:,0] = R[0,:]
rx_positions[:,1] = R[2,:]
rx_positions[:,2] = R[1,:]


from src.utils.data import make_dir
make_dir('data/datasets/experimental/Yarovoy/')
torch.save(MEAS_NCKTR,'data/datasets/experimental/Yarovoy/measurements.pt')
torch.save(F,'data/datasets/experimental/Yarovoy/freqs.pt')
torch.save(tx_positions,'data/datasets/experimental/Yarovoy/tx_positions.pt')
torch.save(rx_positions,'data/datasets/experimental/Yarovoy/rx_positions.pt')


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Number of Transmitters:         16
Number of Receivers:             9
Number of Freqs.:              251
Measurement Shape (NCktr):     (1, 1, 251, 16, 9)


... and you are done. Please refer to "main-experimental.ipynb" for experimental reconstructions.                                           